In [5]:
learning_rate = 0.01
num_steps = 20
batch_size = 20
total_numbers = 291
display_step = 1000
examples_to_show = 10
import tensorflow as tf
# Network Parameters
num_hidden_1 = 256 # 1st layer num features
num_hidden_2 = 128
num_hidden_3 = 64
num_hidden_4 = 32# 2nd layer num features (the latent dim)
num_input = 26 
num_classes = 7

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])


weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'encoder_h3': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_3])),
    'encoder_h4': tf.Variable(tf.random_normal([num_hidden_3, num_hidden_4])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_4, num_hidden_3])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_3, num_hidden_2])),
    'decoder_h3': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h4': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
    'classifier_h': tf.Variable(tf.random_normal([num_hidden_4, num_classes])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'encoder_b3': tf.Variable(tf.random_normal([num_hidden_3])),
    'encoder_b4': tf.Variable(tf.random_normal([num_hidden_4])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_3])),
    'decoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b3': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b4': tf.Variable(tf.random_normal([num_input])),
    'classifier_b': tf.Variable(tf.random_normal([num_classes])),
}

In [6]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['encoder_h3']),
                                   biases['encoder_b3']))
    layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, weights['encoder_h4']),
                                   biases['encoder_b4']))
    
    return layer_4


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    layer_3 = tf.nn.sigmoid(tf.add(tf.matmul(layer_2, weights['decoder_h3']),
                                   biases['decoder_b3']))
    layer_4 = tf.nn.sigmoid(tf.add(tf.matmul(layer_3, weights['decoder_h4']),
                                   biases['decoder_b4']))
    return layer_4

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op

label_pred = tf.nn.softmax(tf.add(tf.matmul(encoder_op, weights['classifier_h']), biases['classifier_b']))
y_clipped = tf.clip_by_value(label_pred, 1e-10, 0.9999999)


# Targets (Labels) are the input data.
y_true = X
label_true = Y

# Define loss and optimizer, minimize the squared error
loss_autoencoder = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
cross_entropy_loss = -tf.reduce_mean(tf.reduce_sum(label_true * tf.log(y_clipped)
                         + (1 - label_true) * tf.log(1 - y_clipped), axis=1))
loss_total = loss_autoencoder+cross_entropy_loss

optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_total)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [7]:
from numpy import genfromtxt
my_data = genfromtxt('./DataForBiplab/2017Kharif_NDVI1.csv', delimiter=',')
my_data1 = genfromtxt('./DataForBiplab/Gamma10m_2017Kharif_Gamma_VV1.csv', delimiter=',')

In [8]:
data = my_data[1:,2:]
data1 = my_data1[1:,2:]
print(data.shape)
print(data1.shape)

(546, 13)
(546, 13)


In [9]:
for i in range (0,data1.shape[0]):
    for j in range (0,data1.shape[1]):
        if (data1[i][j]>1):
            print(data1[i][j])

1.0113443136215199
3.4080424308776895
1.0575135946273801
1.0583688020706201
1.01876056194305
4.5347185134887695
1.08307862281799
1.26069343090057


In [10]:
print(data.max())
print(data1.max())

0.855952701
4.5347185134887695


In [11]:

import numpy as np
data = np.array(data)
data1 = np.array(data1)
print(data.shape)
print(data1.shape)

(546, 13)
(546, 13)


In [12]:
import pandas
colnames = ['Latitude', 'Longitude', 'CropType','SampleYear','ShowingPeriod'
           , 'HarvestingPeriod', 'Others']
data3 = pandas.read_csv('./DataForBiplab/FieldDataKharif2017.csv', names=colnames)

In [13]:
ctype = data3.CropType.tolist()
print(ctype)

['CropType', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Paddy', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcane', 'Sugarcan

In [14]:
p = np.array(ctype)
print(p.shape)

(547,)


In [15]:
lbl1 = ctype
label = []
for i in range(1,547):
    if(lbl1[i] == "Paddy"):
        label += [0]
    if(lbl1[i] == "Sugarcane"):
        label += [1]
    if(lbl1[i] == "PearlMillet"):
        label += [2]
    if(lbl1[i] == "PigeonPea"):
        label += [3]
    if(lbl1[i] == "Sorghum"):
        label += [4]
    if(lbl1[i] == "Forest/NonCropland"):
        label += [5]
    if(lbl1[i] == "Habitation"):
        label += [6]
    
print(label)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 

In [16]:
label = np.array(label)
print(label.shape)

(546,)


In [17]:
print(data.shape)
print(data1.shape)
data2 = np.concatenate((data,data1), axis=1)
print(data2.shape)

(546, 13)
(546, 13)
(546, 26)


In [18]:
x1=[]
x2=[]
x3=[]
x4=[]
x5=[]
x6=[]
x7=[]
l1=[]
l2=[]
l3=[]
l4=[]
l5=[]
l6=[]
l7=[]
for i in range(0, label.shape[0]):
    if(label[i]==0):
        x1.append(data2[i])
        l1.append([1,0,0,0,0,0,0])
    if(label[i]==1):
        x2.append(data2[i])
        l2.append([0,1,0,0,0,0,0])
    if(label[i]==2):
        x3.append(data2[i])
        l3.append([0,0,1,0,0,0,0])
    if(label[i]==3):
        x4.append(data2[i])
        l4.append([0,0,0,1,0,0,0])
    if(label[i]==4):
        x5.append(data2[i])
        l5.append([0,0,0,0,1,0,0])
    if(label[i]==5):
        x6.append(data2[i])
        l6.append([0,0,0,0,0,1,0])
    if(label[i]==6):
        x7.append(data2[i])
        l7.append([0,0,0,0,0,0,1])
x1=np.array(x1)
x2=np.array(x2)
x3=np.array(x3)
x4=np.array(x4)
x5=np.array(x5)
x6=np.array(x6)
x7=np.array(x7)

l1=np.array(l1)
l2=np.array(l2)
l3=np.array(l3)
l4=np.array(l4)
l5=np.array(l5)
l6=np.array(l6)
l7=np.array(l7)

print(l7.shape)
print(x6.shape)

(209, 7)
(34, 26)


In [19]:
from sklearn.utils import shuffle
data2, label = shuffle(data2, label, random_state = 0)

In [20]:
# 80% train 20% test (NDVIclean)
X_train = data2[0:436,:]
X_test = data2[436:,:]
y_train = label[0:436]
y_test = label[436:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(436, 26)
(110, 26)
(436,)
(110,)


In [21]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(436, 7)
(110, 7)


/home/aniruddha/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [22]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 20
num_batch = 12

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 1.48075902
Epoch: 2 cost = 1.29150947
Epoch: 3 cost = 1.16725997
Epoch: 4 cost = 1.04339560
Epoch: 5 cost = 0.92177812
Epoch: 6 cost = 0.81386228
Epoch: 7 cost = 0.72594613
Epoch: 8 cost = 0.65577640
Epoch: 9 cost = 0.59645104
Epoch: 10 cost = 0.54420992
Epoch: 11 cost = 0.50038437
Epoch: 12 cost = 0.46718140
Epoch: 13 cost = 0.44398435
Epoch: 14 cost = 0.42798054
Epoch: 15 cost = 0.41600712
Epoch: 16 cost = 0.40567944
Epoch: 17 cost = 0.39600851
Epoch: 18 cost = 0.38689852
Epoch: 19 cost = 0.37852019
Epoch: 20 cost = 0.37106755
Epoch: 21 cost = 0.36460723
Epoch: 22 cost = 0.35890308
Epoch: 23 cost = 0.35342479
Epoch: 24 cost = 0.34754537
Epoch: 25 cost = 0.34057256
Epoch: 26 cost = 0.33169591
Epoch: 27 cost = 0.32018922
Epoch: 28 cost = 0.30847927
Epoch: 29 cost = 0.31598622
Epoch: 30 cost = 0.34603287
Epoch: 31 cost = 0.27290202
Epoch: 32 cost = 0.26914013
Epoch: 33 cost = 0.28137948
Epoch: 34 cost = 0.45168306
Epoch: 35 cost = 0.30608169
Epoch: 36 cost = 0.38189181
E

Epoch: 324 cost = 0.00778728
Epoch: 325 cost = 0.00752412
Epoch: 326 cost = 0.00744642
Epoch: 327 cost = 0.00797642
Epoch: 328 cost = 0.00721470
Epoch: 329 cost = 0.00718671
Epoch: 330 cost = 0.00697341
Epoch: 331 cost = 0.00697333
Epoch: 332 cost = 0.00669599
Epoch: 333 cost = 0.00644909
Epoch: 334 cost = 0.00664399
Epoch: 335 cost = 0.00683023
Epoch: 336 cost = 0.00640851
Epoch: 337 cost = 0.00695438
Epoch: 338 cost = 0.00763381
Epoch: 339 cost = 0.00693774
Epoch: 340 cost = 0.00676241
Epoch: 341 cost = 0.00602459
Epoch: 342 cost = 0.00601937
Epoch: 343 cost = 0.00613817
Epoch: 344 cost = 0.00631670
Epoch: 345 cost = 0.00761615
Epoch: 346 cost = 0.09970590
Epoch: 347 cost = 0.54117787
Epoch: 348 cost = 0.58858993
Epoch: 349 cost = 0.09962755
Epoch: 350 cost = 0.02315816
Epoch: 351 cost = 0.01131856
Epoch: 352 cost = 0.00871403
Epoch: 353 cost = 0.00872551
Epoch: 354 cost = 0.00873182
Epoch: 355 cost = 0.00778234
Epoch: 356 cost = 0.00830107
Epoch: 357 cost = 0.00798654
Epoch: 358 cos

Epoch: 639 cost = 0.00632208
Epoch: 640 cost = 0.00568221
Epoch: 641 cost = 0.00558643
Epoch: 642 cost = 0.00532262
Epoch: 643 cost = 0.00552549
Epoch: 644 cost = 0.00589583
Epoch: 645 cost = 0.00601263
Epoch: 646 cost = 0.00606971
Epoch: 647 cost = 0.00634371
Epoch: 648 cost = 0.00587384
Epoch: 649 cost = 0.00630088
Epoch: 650 cost = 0.00604777
Epoch: 651 cost = 0.00571763
Epoch: 652 cost = 0.00610217
Epoch: 653 cost = 0.00588184
Epoch: 654 cost = 0.00602807
Epoch: 655 cost = 0.00546052
Epoch: 656 cost = 0.00572373
Epoch: 657 cost = 0.00567821
Epoch: 658 cost = 0.00538262
Epoch: 659 cost = 0.00550552
Epoch: 660 cost = 0.00574531
Epoch: 661 cost = 0.00568047
Epoch: 662 cost = 0.00592313
Epoch: 663 cost = 0.00679760
Epoch: 664 cost = 0.00535033
Epoch: 665 cost = 0.00531224
Epoch: 666 cost = 0.00576124
Epoch: 667 cost = 0.00539594
Epoch: 668 cost = 0.00609206
Epoch: 669 cost = 0.00635643
Epoch: 670 cost = 0.00723488
Epoch: 671 cost = 0.31927360
Epoch: 672 cost = 0.42074473
Epoch: 673 cos

Epoch: 957 cost = 0.00532947
Epoch: 958 cost = 0.00549728
Epoch: 959 cost = 0.00551595
Epoch: 960 cost = 0.00520653
Epoch: 961 cost = 0.00562726
Epoch: 962 cost = 0.00530522
Epoch: 963 cost = 0.00511211
Epoch: 964 cost = 0.00543459
Epoch: 965 cost = 0.00578997
Epoch: 966 cost = 0.00523898
Epoch: 967 cost = 0.00567078
Epoch: 968 cost = 0.00622422
Epoch: 969 cost = 0.01157142
Epoch: 970 cost = 0.00745014
Epoch: 971 cost = 0.00573731
Epoch: 972 cost = 0.00569063
Epoch: 973 cost = 0.00579315
Epoch: 974 cost = 0.00519305
Epoch: 975 cost = 0.00481637
Epoch: 976 cost = 0.00504191
Epoch: 977 cost = 0.00538341
Epoch: 978 cost = 0.00539081
Epoch: 979 cost = 0.00525661
Epoch: 980 cost = 0.00518801
Epoch: 981 cost = 0.00558854
Epoch: 982 cost = 0.00586986
Epoch: 983 cost = 0.00566307
Epoch: 984 cost = 0.00546108
Epoch: 985 cost = 0.00507666
Epoch: 986 cost = 0.00569466
Epoch: 987 cost = 0.00670479
Epoch: 988 cost = 0.30050316
Epoch: 989 cost = 0.20943196
Epoch: 990 cost = 0.06435209
Epoch: 991 cos

In [23]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(label_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [24]:
print(y_test.shape)

(110, 7)


In [25]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.79999995


In [26]:
res1=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x1, Y: l1}))
res2=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x2, Y: l2}))
res3=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x3, Y: l3}))
res4=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x4, Y: l4}))
res5=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x5, Y: l5}))
res6=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x6, Y: l6}))
res7=np.array(sess.run(tf.argmax(label_pred,1), feed_dict={X: x7, Y: l7}))
print(res6)

[5 5 5 5 1 1 1 5 5 1 1 4 5 5 5 5 5 5 1 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]


In [23]:
confusion=[]
x1=np.sum(np.equal(res1, 0))
x2=np.sum(np.equal(res1, 1))
x3=np.sum(np.equal(res1, 2))
x4=np.sum(np.equal(res1, 3))
x5=np.sum(np.equal(res1, 4))
x6=np.sum(np.equal(res1, 5))
x7=np.sum(np.equal(res1, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res2, 0))
x2=np.sum(np.equal(res2, 1))
x3=np.sum(np.equal(res2, 2))
x4=np.sum(np.equal(res2, 3))
x5=np.sum(np.equal(res2, 4))
x6=np.sum(np.equal(res2, 5))
x7=np.sum(np.equal(res2, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res3, 0))
x2=np.sum(np.equal(res3, 1))
x3=np.sum(np.equal(res3, 2))
x4=np.sum(np.equal(res3, 3))
x5=np.sum(np.equal(res3, 4))
x6=np.sum(np.equal(res3, 5))
x7=np.sum(np.equal(res3, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res4, 0))
x2=np.sum(np.equal(res4, 1))
x3=np.sum(np.equal(res4, 2))
x4=np.sum(np.equal(res4, 3))
x5=np.sum(np.equal(res4, 4))
x6=np.sum(np.equal(res4, 5))
x7=np.sum(np.equal(res4, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res5, 0))
x2=np.sum(np.equal(res5, 1))
x3=np.sum(np.equal(res5, 2))
x4=np.sum(np.equal(res5, 3))
x5=np.sum(np.equal(res5, 4))
x6=np.sum(np.equal(res5, 5))
x7=np.sum(np.equal(res5, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res6, 0))
x2=np.sum(np.equal(res6, 1))
x3=np.sum(np.equal(res6, 2))
x4=np.sum(np.equal(res6, 3))
x5=np.sum(np.equal(res6, 4))
x6=np.sum(np.equal(res6, 5))
x7=np.sum(np.equal(res6, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

x1=np.sum(np.equal(res7, 0))
x2=np.sum(np.equal(res7, 1))
x3=np.sum(np.equal(res7, 2))
x4=np.sum(np.equal(res7, 3))
x5=np.sum(np.equal(res7, 4))
x6=np.sum(np.equal(res7, 5))
x7=np.sum(np.equal(res7, 6))
confusion.append([x1,x2,x3,x4,x5,x6,x7])

confusion=np.array(confusion)
print(confusion)

[[ 41  12   1   0   6   0   1]
 [  7 155   2   0   2   6   0]
 [ 13   4   6   0   7   0   3]
 [  1   8   0   0   0   1   0]
 [  3   6   5   0  10   0   3]
 [  0   6   0   0   0  28   0]
 [  1   5   0   0   1   4 198]]


In [42]:
# 70% train 30% test (NDVIclean)
# Train - 204
# Test - 87
X_train = data2[0:382,:]
X_test = data2[382:,:]
y_train = label[0:382]
y_test = label[382:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(382, 27)
(164, 27)
(382,)
(164,)


In [43]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(382, 7)
(164, 7)


In [44]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 20
num_batch = 10

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 1.31621733
Epoch: 2 cost = 1.27310133
Epoch: 3 cost = 1.23308244
Epoch: 4 cost = 1.19668131
Epoch: 5 cost = 1.16403999
Epoch: 6 cost = 1.13444862
Epoch: 7 cost = 1.10652390
Epoch: 8 cost = 1.07880507
Epoch: 9 cost = 1.05023336
Epoch: 10 cost = 1.02030907
Epoch: 11 cost = 0.98903160
Epoch: 12 cost = 0.95676680
Epoch: 13 cost = 0.92408876
Epoch: 14 cost = 0.89159198
Epoch: 15 cost = 0.85969896
Epoch: 16 cost = 0.82855673
Epoch: 17 cost = 0.79809556
Epoch: 18 cost = 0.76821723
Epoch: 19 cost = 0.73897948
Epoch: 20 cost = 0.71067791
Epoch: 21 cost = 0.68381283
Epoch: 22 cost = 0.65898256
Epoch: 23 cost = 0.63674395
Epoch: 24 cost = 0.61747818
Epoch: 25 cost = 0.60129573
Epoch: 26 cost = 0.58800831
Epoch: 27 cost = 0.57717793
Epoch: 28 cost = 0.56822860
Epoch: 29 cost = 0.56057372
Epoch: 30 cost = 0.55370443
Epoch: 31 cost = 0.54722438
Epoch: 32 cost = 0.54086208
Epoch: 33 cost = 0.53448799
Epoch: 34 cost = 0.52809622
Epoch: 35 cost = 0.52171738
Epoch: 36 cost = 0.51534514
E

Epoch: 317 cost = 0.00088412
Epoch: 318 cost = 0.00090036
Epoch: 319 cost = 0.00104986
Epoch: 320 cost = 0.00109031
Epoch: 321 cost = 0.00085428
Epoch: 322 cost = 0.00084826
Epoch: 323 cost = 0.00104231
Epoch: 324 cost = 0.00094844
Epoch: 325 cost = 0.00090704
Epoch: 326 cost = 0.00084045
Epoch: 327 cost = 0.00095803
Epoch: 328 cost = 0.00099999
Epoch: 329 cost = 0.00077827
Epoch: 330 cost = 0.00085815
Epoch: 331 cost = 0.00096030
Epoch: 332 cost = 0.00083618
Epoch: 333 cost = 0.00082010
Epoch: 334 cost = 0.00096529
Epoch: 335 cost = 0.00096386
Epoch: 336 cost = 0.00088184
Epoch: 337 cost = 0.00080761
Epoch: 338 cost = 0.00078367
Epoch: 339 cost = 0.00093282
Epoch: 340 cost = 0.00100300
Epoch: 341 cost = 0.00086980
Epoch: 342 cost = 0.00087183
Epoch: 343 cost = 0.00077055
Epoch: 344 cost = 0.00081367
Epoch: 345 cost = 0.00086376
Epoch: 346 cost = 0.00078789
Epoch: 347 cost = 0.00074560
Epoch: 348 cost = 0.00082273
Epoch: 349 cost = 0.00077811
Epoch: 350 cost = 0.00088807
Epoch: 351 cos

Epoch: 640 cost = 0.00043585
Epoch: 641 cost = 0.00049883
Epoch: 642 cost = 0.00047197
Epoch: 643 cost = 0.00037567
Epoch: 644 cost = 0.00053130
Epoch: 645 cost = 0.00040214
Epoch: 646 cost = 0.00034198
Epoch: 647 cost = 0.00037285
Epoch: 648 cost = 0.00053922
Epoch: 649 cost = 0.00051411
Epoch: 650 cost = 0.00039376
Epoch: 651 cost = 0.00036506
Epoch: 652 cost = 0.00040949
Epoch: 653 cost = 0.00039929
Epoch: 654 cost = 0.00053150
Epoch: 655 cost = 0.00053562
Epoch: 656 cost = 0.00040947
Epoch: 657 cost = 0.00040095
Epoch: 658 cost = 0.00051491
Epoch: 659 cost = 0.00047686
Epoch: 660 cost = 0.00027335
Epoch: 661 cost = 0.00033834
Epoch: 662 cost = 0.00051496
Epoch: 663 cost = 0.00041674
Epoch: 664 cost = 0.00041039
Epoch: 665 cost = 0.00043714
Epoch: 666 cost = 0.00046412
Epoch: 667 cost = 0.00036242
Epoch: 668 cost = 0.00041827
Epoch: 669 cost = 0.00048895
Epoch: 670 cost = 0.00035854
Epoch: 671 cost = 0.00049787
Epoch: 672 cost = 0.00049665
Epoch: 673 cost = 0.00035721
Epoch: 674 cos

Epoch: 949 cost = 0.00036320
Epoch: 950 cost = 0.00028582
Epoch: 951 cost = 0.00037718
Epoch: 952 cost = 0.00030684
Epoch: 953 cost = 0.00038355
Epoch: 954 cost = 0.00036667
Epoch: 955 cost = 0.00029234
Epoch: 956 cost = 0.00027406
Epoch: 957 cost = 0.00024336
Epoch: 958 cost = 0.00030580
Epoch: 959 cost = 0.00026421
Epoch: 960 cost = 0.00032353
Epoch: 961 cost = 0.00033545
Epoch: 962 cost = 0.00034953
Epoch: 963 cost = 0.00043187
Epoch: 964 cost = 0.00038310
Epoch: 965 cost = 0.00032729
Epoch: 966 cost = 0.00026877
Epoch: 967 cost = 0.00027497
Epoch: 968 cost = 0.00029204
Epoch: 969 cost = 0.00024047
Epoch: 970 cost = 0.00028655
Epoch: 971 cost = 0.00035449
Epoch: 972 cost = 0.00029259
Epoch: 973 cost = 0.00037795
Epoch: 974 cost = 0.00042288
Epoch: 975 cost = 0.00026744
Epoch: 976 cost = 0.00033540
Epoch: 977 cost = 0.00045800
Epoch: 978 cost = 0.00026902
Epoch: 979 cost = 0.00026160
Epoch: 980 cost = 0.00032580
Epoch: 981 cost = 0.00025944
Epoch: 982 cost = 0.00032033
Epoch: 983 cos

In [46]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.8170731
